<a href="https://colab.research.google.com/github/ni-metre/Colab_Triplet_Loss_Backup/blob/master/Classifier_Pickle_Maker_with_Testnum_and_Peak_MetaData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
% xmode verbose
#Set up gspread
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

Exception reporting mode: Verbose


In [2]:

#Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# import the necessary packages
import warnings

warnings.filterwarnings("ignore")


#import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pandas as pd
import numpy as np
import glob
import random
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns; sns.set()
import matplotlib.patheffects as PathEffects
import os
from numpy import array
import datetime
from collections import defaultdict
import pickle
import math
from sklearn.metrics import mean_squared_error

In [5]:
#path to fake data
sensor_of_interest = '09'
path = '/content/drive/My Drive/Fakes_' + sensor_of_interest

pickle_path = '/content/drive/My Drive/Pickles'

#path to model checkpoints
checkpoint_path = '/content/drive/My Drive/Checkpoints'

#path to real data
real_path = '/content/drive/My Drive/Test_outputs/General'

In [6]:
test_db_df=pd.read_csv('/content/drive/My Drive/Original_Fake_Test_Log.csv')
#Convert to numeric to enable test sorting/picking
for column in test_db_df[['Acetone', 'CO2', 'H2O', 'Sensor', 'Test_that_day', 'Time_Btwn_s', 'Instr_Temp', 'Baseline', 'File', 'Total CO2']]:
    test_db_df[column] = pd.to_numeric(test_db_df[column])
test_db_df.set_index('File', inplace = True)
test_db_df.head()

,Raw_data,Test_name,H2O,CO2,Total CO2,Acetone,Amb_Temp,Other,Instr,Sensor,Date,Time,Time_Btwn_s,Test_that_day,Rel_Hum,EtOH_temp,Instr_Temp,Instr_T2,Instr_T3,FLAG,Notes,Sample_rate,Data_pts,Baseline,New_Baseline,Baseline_Dip,Peak_neg,Peak_neg_t,Peak_neg_t_ix,Cross_t,Cross_t_ix,Peak_pos,Peak_pos_t,Peak_pos_t_ix,Area_neg_raw,Area_pos_raw,Area_neg,Area_pos_X,Area_total_X,Area_pos,...,Area_to_Cal_int,Area_Pos_to_Cal_int,Baseline_slope,slope_calc,flag_slope,a_,b_,c_,del_a,del_b,del_c,cpn_time,cpn_mV,cpn_manual_mV,cpn_manual_time,solve,solve2,del_pnt,del_ppt,del_xtime,del_cpnt,del_pn,del_pp,slope_pos_peak,slope_pos_peak_t,slope_pos_peak_t_ix,slope_neg_peak,slope_neg_peak_t,slope_neg_peak_t_ix,slope_cross_t,slope_cross_t_ix,dx2_pos_peak,dx2_pos_peak_t,dx2_pos_peak_t_ix,dx2_neg_peak,dx2_neg_peak_t,dx2_neg_peak_t_ix,dx2_cross_t,dx2_cross_t_ix,Area_pos_old
File,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
200000.0,NaN,Combo of 5134 and 5262 | Weight - 0.79,10.0,42100.0,58.98,9.2125,NaN,NaN,38263.0,14.0,NaN,NaN,603.85,19.16,NaN,NaN,26.5967,NaN,NaN,NaN,NaN,250.0,55701.0,26.907357,27.085221,NaN,-395.676821,2.356,589.0,9.056,2264.0,45.245946,14.248,3562.0,NaN,NaN,-1770.0,NaN,NaN,1643.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200001.0,NaN,Combo of 5134 and 5262 | Weight - 0.67,10.0,43300.0,55.14,8.7625,NaN,NaN,38263.0,14.0,NaN,NaN,602.05,18.68,NaN,NaN,27.9491,NaN,NaN,NaN,NaN,250.0,55701.0,27.999765,28.284850,NaN,-427.395069,2.332,583.0,8.900,2225.0,48.576854,14.248,3562.0,NaN,NaN,-1876.0,NaN,NaN,1744.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200002.0,NaN,Combo of 5134 and 5262 | Weight - 0.94,10.0,40600.0,63.78,9.7750,NaN,NaN,38263.0,14.0,NaN,NaN,606.10,19.76,NaN,NaN,24.9062,NaN,NaN,NaN,NaN,250.0,55701.0,25.541849,25.585685,NaN,-356.231984,2.408,602.0,9.276,2319.0,41.082312,14.248,3562.0,NaN,NaN,-1638.0,NaN,NaN,1513.56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200003.0,NaN,Combo of 5134 and 5262 | Weight - 0.52,10.0,44800.0,50.34,8.2000,NaN,NaN,38263.0,14.0,NaN,NaN,599.80,18.08,NaN,NaN,29.6396,NaN,NaN,NaN,NaN,250.0,55701.0,29.365274,29.784387,NaN,-467.191336,2.288,572.0,8.728,2182.0,52.825978,13.876,3469.0,NaN,NaN,-2010.0,NaN,NaN,1866.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200004.0,NaN,Combo of 5134 and 5262 | Weight - 0.52,10.0,44800.0,50.34,8.2000,NaN,NaN,38263.0,14.0,NaN,NaN,599.80,18.08,NaN,NaN,29.6396,NaN,NaN,NaN,NaN,250.0,55701.0,29.365274,29.784387,NaN,-467.191336,2.288,572.0,8.728,2182.0,52.825978,13.876,3469.0,NaN,NaN,-2010.0,NaN,NaN,1866.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def get_test_list (Acetone, CO2, H2O, Sensor, Range1, Range2):
    
    if Sensor == 'all':
        filtered = test_db_df
    else:
        filtered = test_db_df.query('Sensor == @Sensor')
    if Acetone != 'all':
        if str(Acetone)[:1] == '<':
            comp = float(Acetone[1:])
            filtered = filtered.query('Acetone < @comp')
        else:
            if str(Acetone)[:1] == '>':
                comp = float(Acetone[1:])
                filtered = filtered.query('Acetone > @comp')
            else:
                filtered = filtered.query('Acetone == @Acetone')
    if CO2 != 'all':
        if CO2 == '+':
            filtered = filtered.query('CO2 != 0')
        else:
            if CO2 == 4:
                filtered = filtered.query('CO2 >= 40000 & CO2 <= 46666.67')
            else:
                if CO2 == 5:
                    filtered = filtered.query('CO2 > 46666.67 & CO2 <= 53333.33')
                else:
                    filtered = filtered.query('CO2 > 53333.33 & CO2 <= 60000')
    filtered = filtered.query('H2O == @H2O')
    filtered['Tests'] = filtered.index  
    list_of_tests = filtered['Tests'].tolist()
    return([y for y in list_of_tests if ((int(y)>=Range1)&(int(y)<Range2))])

In [9]:
test_list_09 = random.choices(get_test_list('all', '+', 10, 9, 1000, 1000000), k=10000)
test_list_10 = random.choices(get_test_list('all', '+', 10, 10, 1000, 2000000), k=10000)
test_list_14 = random.choices(get_test_list('all', '+', 10, 14, 1000, 1000000), k=10000)
test_list_15 = random.choices(get_test_list('all', '+', 10, 15, 1000, 1000000), k=10000)
test_list_16 = random.choices(get_test_list('all', '+', 10, 16, 1000, 1000000), k=10000)
test_list_19 = random.choices(get_test_list('all', '+', 10, 19, 1000, 2000000), k=10000)

In [11]:
test_list = test_list_09 + test_list_10 + test_list_14 + test_list_15 + test_list_16 + test_list_19
print(len(test_list))

60000


In [ ]:

scale = 1
cut_off_time =30

datadict = defaultdict(list)



testnum_list = []
mV_list = []
Ac_list = []
co2_list = []
temp_list = []
sensor_list = []
metadata_list = []
pp_list = []
ppt_list = []
pn_list = []
pnt_list = []

counter_list = []
for j in test_list:
    ac = float(test_db_df.loc[j, 'Acetone'])
    co2 = int(test_db_df.loc[j, 'CO2'])/2000
    temp = float(test_db_df.loc[j, 'Instr_Temp'])
    sensr = int(test_db_df.loc[j, 'Sensor'])
    sample_rate = int(test_db_df.loc[j, 'Sample_rate'])
    num_of_samples = int(cut_off_time*sample_rate/scale)
    bs = float(test_db_df.loc[j, 'Baseline'])
    timebtwn = float(test_db_df.loc[j, 'Time_Btwn_s'])
    totalco2 = float(test_db_df.loc[j, 'Total CO2'])
    testday = float(test_db_df.loc[j, 'Test_that_day'])
    pp =  float(test_db_df.loc[j, 'Peak_pos'])
    ppt =  float(test_db_df.loc[j, 'Peak_pos_t'])
    pn =  float(test_db_df.loc[j, 'Peak_neg'])
    pnt =  float(test_db_df.loc[j, 'Peak_neg_t'])

    if j < 200000:
      sensor_of_interest = '09'
    elif j >= 200000 and j < 400000:
      sensor_of_interest = '14'
    elif j >= 400000 and j < 600000:
      sensor_of_interest = '15'
    elif j >= 600000 and j < 800000:
      sensor_of_interest = '16'
    elif j >= 800000 and j < 1000000:
      sensor_of_interest = '19'
    elif j >= 1000000
      sensor_of_interest = '10'


    full_data = pd.read_csv(path + '/test_output' + str(int(j)) + '.csv',header = 0)
    sig_data = full_data[['Second','mV_rolling_mean']].truncate(after = cut_off_time*sample_rate-1)
    mV_data = sig_data['mV_rolling_mean'].to_numpy()
    mV_data[np.isnan(mV_data)]=0
    ##All the fake data is currently at 250 Hz so no recasting of the data is needed       
    
    mV_list.append(mV_data)
    testnum_list.append(j)
    Ac_list.append(ac)
    co2_list.append(co2)
    temp_list.append(temp)
    sensor_list.append(sensr)
    metadata_list.append([temp, bs, timebtwn, totalco2, testday])
    pp_list.append(pp)
    ppt_list.append(ppt)
    pn_list.append(pn)
    pnt_list.append(pnt)

    counter_list.append(j)
    if len(counter_list) == 100:
      print('Reached the first 100!')
    elif len(counter_list) == 1000:
      print('Reached 1000!')
    elif len(counter_list) == 3000:
      print('Reached 3000!')
    elif len(counter_list) == 6000:
      print('Reached 6000!')
    elif len(counter_list) == 9000:
      print('Reached 9000!--Home-Stretch!')
    else:
      pass
datadict['testnum'] = np.array(testnum_list)
datadict['csvs'] = np.array(mV_list)
datadict['Ac']= np.array(Ac_list)
datadict['CO2']= np.array(co2_list)
datadict['temp']= np.array(temp_list)
datadict['sensor']= np.array(sensor_list)
datadict['metadata'] = np.array(metadata_list)
datadict['peak_pos'] = np.array(pp_list)
datadict['peak_pos_t'] = np.array(ppt_list)
datadict['peak_neg'] = np.array(pn_list)
datadict['peak_neg_t'] = np.array(pnt_list)

In [19]:
datadict['testnum'] = np.array(testnum_list)
datadict['csvs'] = np.array(mV_list)
datadict['Ac']= np.array(Ac_list)
datadict['CO2']= np.array(co2_list)
datadict['temp']= np.array(temp_list)
datadict['sensor']= np.array(sensor_list)
datadict['metadata'] = np.array(metadata_list)
datadict['peak_pos'] = np.array(pp_list)
datadict['peak_pos_t'] = np.array(ppt_list)
datadict['peak_neg'] = np.array(pn_list)
datadict['peak_neg_t'] = np.array(pnt_list)

In [28]:
print(datadict.keys())
print(len(datadict['csvs'][-1]))

dict_keys(['testnum', 'csvs', 'Ac', 'CO2', 'temp', 'sensor', 'metadata', 'peak_pos', 'peak_pos_t', 'peak_neg', 'peak_neg_t'])
7500


In [22]:
# To save datadict as a pickle



with open(pickle_path +'/classifier_train_fake_data_9_10_14_15_16_19_10000ea_csvs_sensor_metadata_peaks.pickle', 'wb') as handle:
    pickle.dump(datadict, handle, protocol=pickle.HIGHEST_PROTOCOL)